# Use Cases for our Model

## Load Dependencies

In [2]:
#import libraries
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv, dotenv_values
import re
import pickle
import pandas as pd
import numpy as np

In [3]:
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_url = os.getenv("REDIRECT_URL")

In [4]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
#extract uri function
def extract_uri(url):
    pattern = r'(track|album|artist|playlist)/([a-zA-Z0-9]{22})' #spotify uri is a 22 char alphanumeric identifier
    
    # Try to find a match in the URL
    match = re.search(pattern, url)

    # If a match is found, return the Spotify ID
    if match:
        return match.group(2)
    else:
        return None

In [6]:
#load the model
file = pickle.load(open("model.pk1","rb"))
model = file['model']
feature_list = file['features']
scaler = file['scaler']

In [7]:
def preprocess(df):
    from sklearn.preprocessing import MinMaxScaler
    features = df[feature_list]
    scaled_df = scaler.transform(features)
    features = pd.DataFrame(scaled_df, columns=features.columns)
    return features

In [9]:
def get_playlist_tracks(playlist_id):
    track_ids = []
    track_names =[]
    track_artists = []
    tracks = sp.playlist_tracks(playlist_id)['items']
    for track in tracks:
        track_ids.append(track['track']['id'])
        track_names.append(track['track']['name'])
        track_artists.append(track['track']['artists'])
    return track_ids, track_names, track_artists

In [10]:
def get_album_tracks(album_id):
    track_ids = []
    track_names = []
    track_artists = []
    tracks = sp.album_tracks(album_id)['items']
    for track in tracks:
        track_ids.append(track['id'])
        track_names.append(track['name'])
        track_artists.append(track['artists'])
    return track_ids, track_names, track_artists



In [11]:
#partition function
def partition(lst, size):
    split = [lst[i:i+size] for i in range(0,len(lst),size)]
    return split

In [12]:
def get_features(track_ids):
    part = partition(track_ids,100) #list of partitioned tracks
    audio_features = []
    for p in part:
        audio_features.extend(sp.audio_features(p))
    return audio_features

## 1. Take in spotify collection URL and classify the collection

### Enter spotify playlist and get classification

In [13]:
# adding column "energetic"
def is_energetic(x):
    return 'Energetic' if x==1 else 'Not Energetic'


def get_playlist_class(url):
    id = extract_uri(url)
    track_ids, track_names, track_artists = get_playlist_tracks(id)

    #finding first artists
    first_artist = []
    for i in range(0,len(track_artists)):
        first_artist.append(track_artists[i][0]['name'])

    
    #making dataframe
    df = pd.DataFrame(get_features(track_ids))
    df.insert(loc=0, column='track_id', value=track_ids)
    df.insert(loc=1, column='track_name', value=track_names)
    df.insert(loc=2, column='first_artist', value=first_artist)

    #extract features
    features = preprocess(df)
    y_pred = model.predict(features)

    df['pred'] = pd.Series(y_pred)
    df['class'] = df['pred'].apply(lambda x: is_energetic(x))


    # adding column "energetic"


    #we will make a classification report
    energetic_tracks = df[['first_artist','track_name']][df.pred==1]
    prop = len(energetic_tracks)/len(df)

    return {'energetic_tracks': energetic_tracks, 'prop': prop, 'dataframe':df}


    

### Check with a mostly energetic playlist

In [14]:
report = get_playlist_class("https://open.spotify.com/playlist/3lE3qloHpUZbGz0IpT1xZk?si=8e984439c78c4238")
report['dataframe']

,track_id,track_name,first_artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,pred,class
0,2EK48RuuE8HHMDzD2rEDqL,Siren,SUNMI,0.610,0.905,9,-3.686,0,0.0369,0.01420,...,122.012,audio_features,2EK48RuuE8HHMDzD2rEDqL,spotify:track:2EK48RuuE8HHMDzD2rEDqL,https://api.spotify.com/v1/tracks/2EK48RuuE8HH...,https://api.spotify.com/v1/audio-analysis/2EK4...,199134,4,1,Energetic
1,60M8FSYZP8MA0Wy2huOADL,LA DI DA,EVERGLOW,0.477,0.850,3,-1.486,0,0.0911,0.00108,...,163.845,audio_features,60M8FSYZP8MA0Wy2huOADL,spotify:track:60M8FSYZP8MA0Wy2huOADL,https://api.spotify.com/v1/tracks/60M8FSYZP8MA...,https://api.spotify.com/v1/audio-analysis/60M8...,210893,4,1,Energetic
2,3As0OA5B06BxLfADvYyG8L,Stay Tonight,CHUNG HA,0.625,0.831,2,-2.186,0,0.0560,0.20300,...,121.025,audio_features,3As0OA5B06BxLfADvYyG8L,spotify:track:3As0OA5B06BxLfADvYyG8L,https://api.spotify.com/v1/tracks/3As0OA5B06Bx...,https://api.spotify.com/v1/audio-analysis/3As0...,217239,4,1,Energetic
3,7rXcCpIAoOUCydkVDMcoPV,INVU,TAEYEON,0.599,0.816,11,-3.412,0,0.0556,0.09110,...,106.962,audio_features,7rXcCpIAoOUCydkVDMcoPV,spotify:track:7rXcCpIAoOUCydkVDMcoPV,https://api.spotify.com/v1/tracks/7rXcCpIAoOUC...,https://api.spotify.com/v1/audio-analysis/7rXc...,204973,4,1,Energetic
4,37ZtpRBkHcaq6hHy0X98zn,I CAN'T STOP ME,TWICE,0.657,0.880,10,-3.672,0,0.1020,0.12100,...,150.084,audio_features,37ZtpRBkHcaq6hHy0X98zn,spotify:track:37ZtpRBkHcaq6hHy0X98zn,https://api.spotify.com/v1/tracks/37ZtpRBkHcaq...,https://api.spotify.com/v1/audio-analysis/37Zt...,205493,4,1,Energetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,54HsCR7lJJdwxmEnTY1JfF,Dun Dun Dance,OH MY GIRL,0.647,0.957,1,-0.384,0,0.0819,0.16900,...,114.968,audio_features,54HsCR7lJJdwxmEnTY1JfF,spotify:track:54HsCR7lJJdwxmEnTY1JfF,https://api.spotify.com/v1/tracks/54HsCR7lJJdw...,https://api.spotify.com/v1/audio-analysis/54Hs...,220560,4,1,Energetic
96,5i5WNh302rkoveunUdZwG2,Starlight,Dreamcatcher,0.548,0.898,6,-3.945,0,0.0484,0.00244,...,149.970,audio_features,5i5WNh302rkoveunUdZwG2,spotify:track:5i5WNh302rkoveunUdZwG2,https://api.spotify.com/v1/tracks/5i5WNh302rko...,https://api.spotify.com/v1/audio-analysis/5i5W...,199093,4,1,Energetic
97,5z2Zf4ORrWDSSJCgDrhz29,Fiesta,VIVIZ,0.687,0.924,2,-2.443,1,0.0639,0.10600,...,116.954,audio_features,5z2Zf4ORrWDSSJCgDrhz29,spotify:track:5z2Zf4ORrWDSSJCgDrhz29,https://api.spotify.com/v1/tracks/5z2Zf4ORrWDS...,https://api.spotify.com/v1/audio-analysis/5z2Z...,211773,4,1,Energetic
98,4N2pGeYYsA7zMOCkXejMd8,DON′T ASK DON′T TELL,EVERGLOW,0.763,0.828,1,-3.034,0,0.0458,0.06880,...,118.044,audio_features,4N2pGeYYsA7zMOCkXejMd8,spotify:track:4N2pGeYYsA7zMOCkXejMd8,https://api.spotify.com/v1/tracks/4N2pGeYYsA7z...,https://api.spotify.com/v1/audio-analysis/4N2p...,208320,4,1,Energetic


In [15]:
report['energetic_tracks']

,first_artist,track_name
0,SUNMI,Siren
1,EVERGLOW,LA DI DA
2,CHUNG HA,Stay Tonight
3,TAEYEON,INVU
4,TWICE,I CAN'T STOP ME
...,...,...
95,OH MY GIRL,Dun Dun Dance
96,Dreamcatcher,Starlight
97,VIVIZ,Fiesta
98,EVERGLOW,DON′T ASK DON′T TELL


In [16]:
report['prop']

0.98

### Enter Spotify Album and get classification

In [17]:
def get_album_class(url):
    id = extract_uri(url)
    track_ids, track_names, track_artists = get_album_tracks(id)
    #finding first artists
    first_artist = []
    for i in range(0,len(track_artists)):
        first_artist.append(track_artists[i][0]['name'])

    
    #making dataframe
    df = pd.DataFrame(get_features(track_ids))
    df.insert(loc=0, column='track_id', value=track_ids)
    df.insert(loc=1, column='track_name', value=track_names)
    df.insert(loc=2, column='first_artist', value=first_artist)

    #extract features
    features = preprocess(df)
    y_pred = model.predict(features)

    df['pred'] = pd.Series(y_pred)
    df['class'] = df['pred'].apply(lambda x: is_energetic(x))


    # adding column "energetic"


    #we will make a classification report
    energetic_tracks = df[['first_artist','track_name']][df.pred==1]
    prop = len(energetic_tracks)/len(df)

    return {'energetic_tracks': energetic_tracks, 'prop': prop, 'dataframe':df}

    
    

#### Check with a mostly energetic album

In [18]:
album = get_album_class("https://open.spotify.com/album/6al2VdKbb6FIz9d7lU7WRB?si=bwvAuuaoRlilrZRox1Ys_A")

In [19]:
album['dataframe'].head()

,track_id,track_name,first_artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,pred,class
0,1IthE5GNiRzFN5CVaCa445,Born Singer,BTS,0.602,0.841,8,-3.334,0,0.109,0.035400,...,157.937,audio_features,1IthE5GNiRzFN5CVaCa445,spotify:track:1IthE5GNiRzFN5CVaCa445,https://api.spotify.com/v1/tracks/1IthE5GNiRzF...,https://api.spotify.com/v1/audio-analysis/1Ith...,238628,4,1,Energetic
1,27S8iOXD7Z58yvJtyk2S9j,No More Dream,BTS,0.438,0.864,2,-5.185,1,0.470,0.011800,...,167.898,audio_features,27S8iOXD7Z58yvJtyk2S9j,spotify:track:27S8iOXD7Z58yvJtyk2S9j,https://api.spotify.com/v1/tracks/27S8iOXD7Z58...,https://api.spotify.com/v1/audio-analysis/27S8...,222067,4,1,Energetic
2,2GEnvQgSJhedm2sqZlOP8o,N.O,BTS,0.597,0.909,11,-4.500,0,0.160,0.044300,...,146.870,audio_features,2GEnvQgSJhedm2sqZlOP8o,spotify:track:2GEnvQgSJhedm2sqZlOP8o,https://api.spotify.com/v1/tracks/2GEnvQgSJhed...,https://api.spotify.com/v1/audio-analysis/2GEn...,209752,4,1,Energetic
3,0vMk4IrUfSJQkhwZnVX6us,Boy In Luv,BTS,0.492,0.931,9,-3.837,0,0.297,0.002440,...,166.071,audio_features,0vMk4IrUfSJQkhwZnVX6us,spotify:track:0vMk4IrUfSJQkhwZnVX6us,https://api.spotify.com/v1/tracks/0vMk4IrUfSJQ...,https://api.spotify.com/v1/audio-analysis/0vMk...,231384,4,1,Energetic
4,0Q53fuiKLGjDKD7Mme7EoQ,Danger,BTS,0.734,0.848,1,-4.433,1,0.102,0.000913,...,112.021,audio_features,0Q53fuiKLGjDKD7Mme7EoQ,spotify:track:0Q53fuiKLGjDKD7Mme7EoQ,https://api.spotify.com/v1/tracks/0Q53fuiKLGjD...,https://api.spotify.com/v1/audio-analysis/0Q53...,245810,4,1,Energetic


In [20]:
album['energetic_tracks']

,first_artist,track_name
0,BTS,Born Singer
1,BTS,No More Dream
2,BTS,N.O
3,BTS,Boy In Luv
4,BTS,Danger
5,BTS,I NEED U
6,BTS,RUN
7,BTS,Burning Up (FIRE)
8,BTS,Blood Sweat & Tears
9,BTS,Spring Day


In [21]:
album['prop']

0.8285714285714286

#### Check with a non energetic album

In [22]:
album2 = get_album_class("https://open.spotify.com/album/12UGkXT4z3ajgq6xheCGDk?si=0lHB1uVCT06mxtHGZDUpoQ")
album2['dataframe']

,track_id,track_name,first_artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,pred,class
0,7cs7MLtAWvsdf3w5kBvfK2,Nothing's Gonna Hurt You Baby,Cigarettes After Sex,0.508,0.330,4,-14.071,1,0.0265,0.259,...,96.818,audio_features,7cs7MLtAWvsdf3w5kBvfK2,spotify:track:7cs7MLtAWvsdf3w5kBvfK2,https://api.spotify.com/v1/tracks/7cs7MLtAWvsd...,https://api.spotify.com/v1/audio-analysis/7cs7...,286293,4,0,Not Energetic
1,73ptBTOgUcD3Wyrq0B9eAW,I'm a Firefighter,Cigarettes After Sex,0.361,0.300,7,-13.657,1,0.0335,0.483,...,114.941,audio_features,73ptBTOgUcD3Wyrq0B9eAW,spotify:track:73ptBTOgUcD3Wyrq0B9eAW,https://api.spotify.com/v1/tracks/73ptBTOgUcD3...,https://api.spotify.com/v1/audio-analysis/73pt...,271019,4,0,Not Energetic
2,5DZNSYxhFA1WA9WEJg9TNY,Dreaming of You,Cigarettes After Sex,0.298,0.398,7,-14.911,1,0.0365,0.549,...,64.135,audio_features,5DZNSYxhFA1WA9WEJg9TNY,spotify:track:5DZNSYxhFA1WA9WEJg9TNY,https://api.spotify.com/v1/tracks/5DZNSYxhFA1W...,https://api.spotify.com/v1/audio-analysis/5DZN...,322219,3,0,Not Energetic
3,3YL4zMqcyFu0OfLI1sZ66Z,Starry Eyes,Cigarettes After Sex,0.419,0.362,2,-15.320,1,0.0337,0.453,...,100.264,audio_features,3YL4zMqcyFu0OfLI1sZ66Z,spotify:track:3YL4zMqcyFu0OfLI1sZ66Z,https://api.spotify.com/v1/tracks/3YL4zMqcyFu0...,https://api.spotify.com/v1/audio-analysis/3YL4...,206000,4,0,Not Energetic


In [23]:
album2['prop']

0.0

## 2. Find energetic songs from Spotify collections & generate a playlist
- Playlist
- Albums
- Artist Discographies

## Other thoughts:

One can also use Oauth 2.0 Authorization flow for getting user's private content's access tokens from Spotify WEB API. This process is simple but require us to use Flask and HTML to provide server functionalities. With that, one can develop this additional functionality:

####  Find energetic songs from User collections & generate a playlist
- User Liked Songs
- User Playlists
- User Listening History